# Extracción de preguntas frecuentes y respuestas sobre matrícula

In [1]:
from bs4 import BeautifulSoup
import requests
import nlpaug.augmenter.word as naw

In [2]:
url = "https://www.uv.es/uvweb/universidad/es/estudios-grado/matricula/matricula/consideraciones-generales/preguntas-frecuentes-1286132590309.html"
r = requests.get(url)

In [3]:
soup = BeautifulSoup(r.text, "html.parser")

In [4]:
s = soup.find("ul", {"class": "bloque-preguntas bp1286132590573"})
lines = s.find_all("li", {"class": "pregunta"})

In [5]:
lines[0]

<li class="pregunta" id="12861326799141286132590573">
<a href="javascript:desplegar('12861326799141286132590573')" style="text-decoration:none">
<h4>Si no consigo plaza en mi primera opción para la Universidad, ¿tengo que matricularme igual?</h4></a>
<span class="fa fa-chevron-down lnktitleFaq12861326799141286132590573"></span>
<div class="bloque-contenido">
<div class="respuesta respuesta-lista"><p style="text-align: justify;">Todos los estudiantes admitidos para cursar estudios en la UV, independientemente del orden de preferencia de la preinscripción, deberán matricularse en el plazo estipulado si no quieren perder la plaza obtenida.</p>
</div>
</div>
</li>

In [6]:
faq_matricula = {}

for line in lines:
    pregunta = line.find("h4").text
    respuesta = line.find("p").text
    faq_matricula[pregunta] = respuesta

In [7]:
faq_matricula

{'Si no consigo plaza en mi primera opción para la Universidad, ¿tengo que matricularme igual?': 'Todos los estudiantes admitidos para cursar estudios en la UV, independientemente del orden de preferencia de la preinscripción, deberán matricularse en el plazo estipulado si no quieren perder la plaza obtenida.',
 '¿Qué pasa si me matriculo en la plaza obtenida, y posteriormente por lista de espera, me adjudican la titulación que yo quiero?': 'Todos los estudiantes a los que se les ha asignado una plaza, y no era la primera solicitada, deberán matricularse en la titulación obtenida para no perder la plaza adjudicada. Si continúan en lista de espera y en algún llamamiento se les adjudica una plaza diferente a la obtenida, deben dirigirse al Centro en el que se han matriculado para anular la matrícula y dirigirse al Centro al que está adscrita la nueva titulación para realizar una nueva matrícula. Más información: http://links.uv.es/2FZ8evO.',
 '¿Qué vale una matrícula en la Universidad?':

## Data augmentation para obtener más frases de entrenamiento

Esta información la queremos añadir como frases de entrenamiento y respuestas del intent faq_matricula.

Sin embargo, cada intent debe tener un mínimo de dos frases de entrenamiento y actualmente solo se tiene una.

Para solucionarlo utilizaremos *Back Translation* una técnica de *Data Augmentation* que consiste en traducir la frase original a otro idioma y luego de vuelva al español para obtener una frase diferente con el mismo significado.

En este caso utilizaremos los idiomas inglés, alemán, francés e italiano para realizar este aumento. Con la frase original tendremos un total de cinco frases de entrenamiento para cada intent.

In [8]:
back_translation_aug_en = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-en",
    to_model_name="Helsinki-NLP/opus-mt-en-es",
)

In [9]:
back_translation_aug_de = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-de",
    to_model_name="Helsinki-NLP/opus-mt-de-es",
)

In [10]:
back_translation_aug_fr = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-fr",
    to_model_name="Helsinki-NLP/opus-mt-fr-es",
)

In [11]:
back_translation_aug_it = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-it",
    to_model_name="Helsinki-NLP/opus-mt-it-es",
)

## Cambiar estructura de los datos para utilizarlos como frases de entrenamiento y respuestas

Ahora transformaremos los datos y los guardaremos en ficheros de texto con la estructura necesaria para poder utilizarlos en el chatbot.

In [12]:
nlu = []
responses = []

for n, (pregunta, respuesta) in enumerate(faq_matricula.items()):
    pregunta_yml = "- intent: faq_matricula/" + str(n) + "\n  examples: |"
    pregunta_yml += "\n    - ".join(
        [
            "",
            pregunta,
            back_translation_aug_en.augment(pregunta),
            back_translation_aug_de.augment(pregunta),
            back_translation_aug_fr.augment(pregunta),
            back_translation_aug_it.augment(pregunta),
        ]
    )
    nlu.append(pregunta_yml)
    respuesta_yml = (
        "  utter_faq_matricula/" + str(n) + ':\n  - text: "' + respuesta + '"'
    )
    responses.append(respuesta_yml)

In [13]:
nlu

['- intent: faq_matricula/0\n  examples: |\n    - Si no consigo plaza en mi primera opción para la Universidad, ¿tengo que matricularme igual?\n    - Si no consigo un lugar en mi primera elección para la universidad, ¿tengo que inscribirme de todos modos?\n    - Si no tengo lugar en mi primera elección para la universidad, ¿tengo que registrarme ahora mismo?\n    - Si no tengo lugar en mi primera opción para la universidad, ¿tengo que inscribirme de la misma manera?\n    - Si no encuentro un lugar en mi primera elección para la universidad, ¿tengo que registrarme de todos modos?',
 '- intent: faq_matricula/1\n  examples: |\n    - ¿Qué pasa si me matriculo en la plaza obtenida, y posteriormente por lista de espera, me adjudican la titulación que yo quiero?\n    - ¿Qué pasa si me inscribo en la posición obtenida, y luego por lista de espera, me otorgan el grado que quiero?\n    - ¿Qué pasa si me inscribo en el lugar que he recibido y luego me dan el título que quiero en la lista de esper

In [14]:
responses

['  utter_faq_matricula/0:\n  - text: "Todos los estudiantes admitidos para cursar estudios en la UV, independientemente del orden de preferencia de la preinscripción, deberán matricularse en el plazo estipulado si no quieren perder la plaza obtenida."',
 '  utter_faq_matricula/1:\n  - text: "Todos los estudiantes a los que se les ha asignado una plaza, y no era la primera solicitada, deberán matricularse en la titulación obtenida para no perder la plaza adjudicada. Si continúan en lista de espera y en algún llamamiento se les adjudica una plaza diferente a la obtenida, deben dirigirse al Centro en el que se han matriculado para anular la matrícula y dirigirse al Centro al que está adscrita la nueva titulación para realizar una nueva matrícula. Más información: http://links.uv.es/2FZ8evO."',
 '  utter_faq_matricula/2:\n  - text: "El coste de la matrícula para estudios oficiales (grados, másteres y doctorados) está limitado por precios públicos . El importe anual de las tasas académicas

In [15]:
with open(r"..\data\internal\nlu_faq_matricula.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(nlu))
with open(
    r"..\data\internal\responses_faq_matricula.txt", "w", encoding="utf-8"
) as f:
    f.write("\n".join(responses))